# Multi-Client Example
*BeamNG.tech / BeamNG.drive*

## Introduction

This example shows how to connect multiple `BeamNGpy` instances to the simulator and have them control different vehicles. For demonstration purposes, both instances are housed in the same Python process, but the same example could be used from an entirely different process or machine as well.

## Scenario

The scenario will be a rather simple one: there are two vehicles on the `smallgrid` map (an infinite flat plane) and one vehicle will randomly drive around, controlled by client A, with another vehicle, controlled by client B, mimicking A's behavior.

## Setup

Contrary to other examples, we will be using two instances of the `BeamNGpy` class representing two clients A and B. In our case, client A will create the scenario containing both vehicles, but client B will later connect and control one of the vehicles while A controls the other. The classes involved in this are:

* `BeamNGpy`: Two instances of this will be used to implement client A and B
* `Scenario`: Client A will create a scenario for both clients to play in
* `Vehicle`: Used to specify and control vehicles involved in the scenario
* `Electrics`: A sensor used to inspect interal systems of the vehicle

The code starts with importing the respective classes:

In [8]:
import random

from beamngpy import BeamNGpy, Scenario, Vehicle
from beamngpy.sensors import Electrics

Now we set up our first client who will create the scenario for both clients. Client A will also launch the simulator.

In [9]:
client_a = BeamNGpy('localhost', 64256, 'D:\BeamNG.tech.v0.27.2.0')
client_a.open()

With the simulator running, we can start setting up our scenario. It will contain two vehicles, the one controlled by Client A being placed in front of the one later controlled by client B.

In [10]:
scenario = Scenario('smallgrid', 'tag')
av_a = Vehicle('vehicleA', model='etk800')
av_b = Vehicle('vehicleB', model='etk800')
scenario.add_vehicle(av_a, pos=(0, -10, 0))
scenario.add_vehicle(av_b)
scenario.make(client_a)

## Running

The scenario is now made, meaning the required files have been generated and can be loaded in the simulator:

In [11]:
client_a.scenario.load(scenario)
client_a.scenario.start()

Now we will set up our second client and connect it to the running simulator. The client will first connect, then query the running scenario, and retrieve currently active vehicles. They will then find the vehicle meant for Client B and connect to it. Note that `client_b` is opened with the flag `launch=False`meaning it will not launch its own BeamNG.tech process.

In [12]:
client_b = BeamNGpy('localhost', 64256)
client_b.open(launch=False)
running_scenario = client_b.scenario.get_current()
print(running_scenario.name)
active_vehicles = client_b.vehicles.get_current()
bv_a = active_vehicles['vehicleA']
bv_b = active_vehicles['vehicleB']
# B attaches their own sensor to get the current controls of A
bv_a.sensors.attach('electrics', Electrics())
# bv_b.sensors.attach('electrics', Electrics())
bv_a.connect(client_b)
bv_b.connect(client_b)

tag


Two clients are now connected to the running simulation and both vehicles. What follows is the main loop of the scenario, where Client A sends random steering inputs to their vehicle and Client B checks how A's vehicle is driving using the electrics sensor and sends the same inputs to their vehicle.

In [13]:
# Focus simulator on second vehicle because it's the more interesting one to see
av_b.focus()

for _ in range(20):
    # Client A sending control inputs to their vehicle connection
    av_a.control(throttle=random.random(), steering=(random.random() * 2 - 1))
    
    # Client B updating the electrics sensor of A's vehicle
    bv_a.sensors.poll()
    
    throttle = bv_a.sensors['electrics']['throttle_input']
    steering = bv_a.sensors['electrics']['steering_input']
    print(throttle, steering)
    bv_b.control(throttle=throttle, steering=steering)
    
    # Client A now advancing the simulation 60 steps
    client_a.control.step(60)

client_b.disconnect()
client_a.close()

0.9397760648495722 -0.027916310377083375
0.0033369488375666334 -0.9059539906442342
0.0033093135845539967 0.06747759350641645
0.6732617804481296 0.36219794860832205
0.7114798575037589 -0.07503945083531902
0.1820544644668287 -0.5304140365940094
0.35971621739410786 0.48805198008987805
0.4804402564387684 0.7119060743514573
0.006492352247964184 -0.31553057626683917
0.7124131473374311 -0.5057307849429381
0.16746279441796375 -0.4040368026482489
0.7165468541861448 -0.2957872971153035
0.1650807385365941 0.7063641741045409
0.8462293316543527 0.6318037202615335
0.5355257334762912 -0.0832615729093024
0.6949452596808949 -0.3248375543566537
0.750583719331257 -0.5069930978730359
0.5553047149758604 0.6663133281224056
0.8697652020016521 0.5200695287843291
0.625095442702823 -0.302663793947006
